In [1]:
import os
import numpy as np

data_dir = "/Users/svenschnydrig/Documents/Coding Challenge/data/EuroSAT_MS_NPY_wo_B10_ordered"
class_names = sorted(os.listdir(data_dir))

x = []
y = []
for i, class_name in enumerate(class_names):
    print(class_name)
    print(i)
    class_dir = os.path.join(data_dir, class_name)
    print(class_dir)
    for filename in os.listdir(class_dir):
        print(filename)
        filepath = os.path.join(class_dir, filename)
        print(filename)
        data = np.load(filepath)
        x.append(data)
        y.append(i)
x = np.stack(x, axis=0)
y = np.array(y)



AnnualCrop
0
/Users/svenschnydrig/Documents/Coding Challenge/data/EuroSAT_MS_NPY_wo_B10_ordered/AnnualCrop
AnnualCrop_1906.npy
AnnualCrop_1906.npy
AnnualCrop_518.npy
AnnualCrop_518.npy
AnnualCrop_1912.npy
AnnualCrop_1912.npy
AnnualCrop_242.npy
AnnualCrop_242.npy
AnnualCrop_1084.npy
AnnualCrop_1084.npy
AnnualCrop_524.npy
AnnualCrop_524.npy
AnnualCrop_2599.npy
AnnualCrop_2599.npy
AnnualCrop_530.npy
AnnualCrop_530.npy
AnnualCrop_1090.npy
AnnualCrop_1090.npy
AnnualCrop_256.npy
AnnualCrop_256.npy
AnnualCrop_2214.npy
AnnualCrop_2214.npy
AnnualCrop_2572.npy
AnnualCrop_2572.npy
AnnualCrop_2566.npy
AnnualCrop_2566.npy
AnnualCrop_1709.npy
AnnualCrop_1709.npy
AnnualCrop_2200.npy
AnnualCrop_2200.npy
AnnualCrop_1721.npy
AnnualCrop_1721.npy
AnnualCrop_2228.npy
AnnualCrop_2228.npy
AnnualCrop_281.npy
AnnualCrop_281.npy
AnnualCrop_1047.npy
AnnualCrop_1047.npy
AnnualCrop_1053.npy
AnnualCrop_1053.npy
AnnualCrop_295.npy
AnnualCrop_295.npy
AnnualCrop_1735.npy
AnnualCrop_1735.npy
AnnualCrop_726.npy
AnnualCr

# Important the order of bands is:
### B1 - 0, B2 - 1, B3 - 2, B4 - 3, B5 - 4, B6 - 5, B7 - 6, B8 - 7, B8A - 8, B9 - 9, B11 - 10, B12 - 11, NDVI - 12, EVI - 13, NDWI - 14, NDBI - 15, NDSI - 16, SAVI - 17, MNDWI - 18, TCB - 19


In [17]:
import tensorflow as tf

def create_tf_dataset(x, y, batch_size=32, shuffle=False): # maybe it would be better to set shuffle to True
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    if shuffle:
        dataset = dataset.shuffle(len(x))
    dataset = dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

# Create the dataset
x = x.astype("float32")  # It's a good practice to convert the data to float32 for better compatibility with TensorFlow
batch_size = 32
tf_dataset = create_tf_dataset(x, y, batch_size, shuffle=False)

In [23]:
tf.data.experimental.cardinality(tf_dataset)

<tf.Tensor: shape=(), dtype=int64, numpy=844>

## Sanity checks

In [18]:
# Get the first element and label from the TensorFlow dataset
first_element_tf, first_label_tf = next(iter(tf_dataset))
print("First element (TensorFlow):", first_element_tf.numpy()[0])  # Convert the tensor to a numpy array and print the first element
print("First label (TensorFlow):", first_label_tf.numpy()[0])  # Convert the tensor to a numpy array and print the first label

First element (TensorFlow): [[[1448. 1448. 1437. ... 1257. 1259. 1263.]
  [1448. 1448. 1437. ... 1257. 1259. 1263.]
  [1425. 1425. 1414. ... 1258. 1261. 1264.]
  ...
  [1257. 1257. 1255. ... 1386. 1381. 1376.]
  [1259. 1259. 1257. ... 1387. 1382. 1376.]
  [1260. 1260. 1258. ... 1388. 1382. 1376.]]

 [[1541. 1541. 1328. ... 1164. 1139. 1134.]
  [1541. 1541. 1328. ... 1164. 1139. 1134.]
  [1173. 1173. 1143. ... 1163. 1147. 1169.]
  ...
  [1051. 1051. 1080. ... 1344. 1357. 1330.]
  [1077. 1077. 1083. ... 1340. 1345. 1344.]
  [1085. 1085. 1091. ... 1328. 1331. 1336.]]

 [[1836. 1836. 1459. ... 1200. 1164. 1192.]
  [1836. 1836. 1459. ... 1200. 1164. 1192.]
  [1244. 1244. 1203. ... 1220. 1165. 1191.]
  ...
  [1105. 1105. 1144. ... 1413. 1420. 1418.]
  [1127. 1127. 1163. ... 1406. 1435. 1434.]
  [1140. 1140. 1138. ... 1401. 1419. 1424.]]

 ...

 [[1063. 1063. 1045. ...  827.  826.  825.]
  [1063. 1063. 1045. ...  827.  826.  825.]
  [1032. 1032. 1015. ...  828.  827.  826.]
  ...
  [ 695.  69

In [20]:
# Get the 3001st element and label from the TensorFlow dataset
index_3001 = 2999  # Index of the 3001st element (0-based indexing)
batch_size = 32

# Calculate which batch the 3001st element belongs to and its index within that batch
batch_index = index_3001 // batch_size
element_index = index_3001 % batch_size

# Get the batch containing the 3001st element
for i, (elements, labels) in enumerate(tf_dataset):
    if i == batch_index:
        element_3001 = elements[element_index].numpy()
        label_3001 = labels[element_index].numpy()
        break

print("3001st element (TensorFlow):", element_3001)
print("3001st label (TensorFlow):", label_3001)

3001st element (TensorFlow): [[[1339. 1339. 1341. ... 1336. 1335. 1335.]
  [1339. 1339. 1341. ... 1336. 1335. 1335.]
  [1342. 1342. 1343. ... 1335. 1334. 1334.]
  ...
  [1311. 1311. 1308. ... 1319. 1319. 1320.]
  [1313. 1313. 1310. ... 1317. 1317. 1318.]
  [1314. 1314. 1312. ... 1316. 1316. 1317.]]

 [[1025. 1025. 1056. ... 1046. 1105. 1097.]
  [1025. 1025. 1056. ... 1046. 1105. 1097.]
  [1029. 1029. 1064. ... 1051. 1073. 1097.]
  ...
  [1024. 1024. 1006. ... 1025. 1018. 1020.]
  [1021. 1021. 1024. ... 1025. 1010. 1008.]
  [1043. 1043. 1025. ... 1021. 1005. 1005.]]

 [[1007. 1007. 1038. ... 1036. 1152. 1152.]
  [1007. 1007. 1038. ... 1036. 1152. 1152.]
  [ 979.  979. 1028. ... 1022. 1084. 1145.]
  ...
  [1026. 1026. 1025. ...  937.  941.  943.]
  [1017. 1017. 1016. ...  939.  941.  942.]
  [1067. 1067. 1044. ...  931.  939.  936.]]

 ...

 [[1105. 1105. 1065. ... 1027. 1038. 1044.]
  [1105. 1105. 1065. ... 1027. 1038. 1044.]
  [1107. 1107. 1066. ... 1010. 1019. 1024.]
  ...
  [1036. 10

In [9]:
import tensorflow as tf


dataset = tf.data.Dataset.from_tensor_slices((x, y))

Metal device set to: Apple M1 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2023-05-02 00:22:37.468882: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-02 00:22:37.468906: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
gpu = len(tf.config.list_physical_devices('GPU'))>0
print ("GPU is","available" if gpu else "NOT AVAILABLE")

GPU is available
